In [1]:
import sys
import os
# Get the current directory and parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
sys.path.insert(0, current_dir)

# Import from the-machine module
import importlib.util
the_machine_path = os.path.join(parent_dir, "the-machine")
sys.path.insert(0, the_machine_path)

# Import all from the-machine and custom_matroid_functions
from HM_module import *
from custom_matroid_functions import *

In [2]:
P = LaurentPolynomialRing(ZZ, 'x')
x = P.gen()
n = 6
M = matroids.CompleteGraphic(n)
r = M.rank()
rank_func = lambda x : M.rank(x)

#e = next(x for x in M.groundset() if x  not in M.coloops())
e = list(M.groundset())[0]
print(M.groundset())
print("deleting ", e)
mdele = M.delete(e)
Eq_M = create_deformed_algebra(M, base_ring=P)
Eq_mdele = create_deformed_algebra(mdele, base_ring=P)
E_mdele = Eq_mdele.natural()

E = Eq_M.natural()
E_basis = E.basis()
E_mdele_basis = E_mdele.basis()

C = Eq_M.C()
C_dele = Eq_mdele.C()
C_basis = C.basis()
C_dele_basis = C_dele.basis()

Z = Eq_M.zeta()
Z_basis = Z.basis()

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})
deleting  0


In [3]:
print("rank of M: ", r)
print("chow polynomial of M: ", chow_polynomial(M))

rank of M:  5
chow polynomial of M:  x^4 + 187*x^3 + 732*x^2 + 187*x + 1


In [4]:
rev = {value: key for key, value in E_basis.items()}

def delta_flat(F):
    F_del_e = frozenset(set(F) - {e})
    return Eq_mdele(x^( - M.rank(F) + mdele.rank(F_del_e)) * E_mdele_basis[F_del_e])
    
def delta(elem):
    rep = E(elem)
    decom = {monomial: coeff for monomial, coeff in zip(rep.monomials(), rep.coefficients())}
    return Eq_mdele(sum(decom[F] * delta_flat(rev[F]) for F in decom))

dict = {F : C_dele(delta(C[F])) for F in E_basis.keys()}

In [5]:
def flat_division(M, e):
    L = M.lattice_of_flats()
    flat_div = {'S':set(), 'T':set(), 'A':set(), 'B':set()}
    for F in L:
        if e in F:
            if F - {e} in L:
                flat_div['T'].add(F)
            else:
                flat_div['A'].add(F)
        else:
            if F.union({e}) in L:
                flat_div['S'].add(F)
            else:
                flat_div['B'].add(F)
    return flat_div

In [6]:
from IPython.display import HTML, display
import pandas as pd

# Get the flat division
flat_div = flat_division(M, e)

# Create separate data lists for each category
data_s = []
data_t = []
data_a = []
data_b = []

basis_list_mdele = list(mdele.lattice_of_flats())

for F in dict:
    elem = dict[F]
    coeff_empty = elem.monomial_coefficients().get(frozenset(), 0)
    
    terms = []
    for S, coeff in elem.monomial_coefficients().items():
        terms.append(f"({coeff})*C[{str(set(S))}]")
    formatted_expr = " + ".join(terms) if terms else "0"
    
    row_data = {
        'F': str(set(F)),
        'rk(F)': M.rank(F),
        'delta(C[F])': formatted_expr,
        'coeff of empty': coeff_empty,
        'in natural basis': str(E_mdele(elem))
    }
    
    # Categorize the flat and add to appropriate list
    if F in flat_div['S']:
        data_s.append(row_data)
    elif F in flat_div['T']:
        data_t.append(row_data)
    elif F in flat_div['A']:
        data_a.append(row_data)
    elif F in flat_div['B']:
        data_b.append(row_data)

# Create and display tables for each category
def display_table(data, category_name, color='lightblue'):
    if data:
        df = pd.DataFrame(data)
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} ({len(data)} flats)</h3>'))
        display(HTML(
            f'<div style="height: 300px; overflow-y: auto; border: 2px solid {color}; padding: 10px;">{df.to_html(index=False)}</div>'
        ))
    else:
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} (0 flats)</h3>'))
        display(HTML('<p>No flats in this category.</p>'))

# Display all four tables
display_table(data_s, 'S', 'green')
display_table(data_t, 'T', 'coral') 
display_table(data_a, 'A', 'blue')
display_table(data_b, 'B', 'red')

F,rk(F),delta(C[F]),coeff of empty,in natural basis
set(),0,(1)*C[set()],1,E[frozenset()]
{10},1,(1)*C[{10}],0,-x*E[frozenset()] + E[frozenset({10})]
{13},1,(1)*C[{13}],0,-x*E[frozenset()] + E[frozenset({13})]
"{10, 13}",2,"(1)*C[{10, 13}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({10})] - x*E[frozenset({13})] + E[frozenset({10, 13})]"
{12},1,(1)*C[{12}],0,-x*E[frozenset()] + E[frozenset({12})]
{14},1,(1)*C[{14}],0,-x*E[frozenset()] + E[frozenset({14})]
"{12, 13, 14}",2,"(1)*C[{12, 13, 14}]",0,"-(1-2*x^2)*E[frozenset()] - x*E[frozenset({13})] - x*E[frozenset({12})] - x*E[frozenset({14})] + E[frozenset({12, 13, 14})]"
{11},1,(1)*C[{11}],0,-x*E[frozenset()] + E[frozenset({11})]
"{10, 11, 14}",2,"(1)*C[{10, 11, 14}]",0,"-(1-2*x^2)*E[frozenset()] - x*E[frozenset({10})] - x*E[frozenset({14})] - x*E[frozenset({11})] + E[frozenset({10, 11, 14})]"
"{11, 12}",2,"(1)*C[{11, 12}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({12})] - x*E[frozenset({11})] + E[frozenset({11, 12})]"


F,rk(F),delta(C[F]),coeff of empty,in natural basis
{0},1,(x^-1 - x)*C[set()],x^-1 - x,(x^-1-x)*E[frozenset()]
"{0, 10}",2,(-1)*C[set()] + (x^-1 - x)*C[{10}],-1,-(2-x^2)*E[frozenset()] + (x^-1-x)*E[frozenset({10})]
"{0, 13}",2,(-1)*C[set()] + (x^-1 - x)*C[{13}],-1,-(2-x^2)*E[frozenset()] + (x^-1-x)*E[frozenset({13})]
"{0, 10, 13}",3,"(-x^-1 - x)*C[set()] + (-1)*C[{10}] + (-1)*C[{13}] + (x^-1 - x)*C[{10, 13}]",-x^-1 - x,"-(2*x^-1-3*x+x^3)*E[frozenset()] - (2-x^2)*E[frozenset({10})] - (2-x^2)*E[frozenset({13})] + (x^-1-x)*E[frozenset({10, 13})]"
"{0, 12}",2,(-1)*C[set()] + (x^-1 - x)*C[{12}],-1,-(2-x^2)*E[frozenset()] + (x^-1-x)*E[frozenset({12})]
"{0, 14}",2,(-1)*C[set()] + (x^-1 - x)*C[{14}],-1,-(2-x^2)*E[frozenset()] + (x^-1-x)*E[frozenset({14})]
"{0, 12, 13, 14}",3,"(-x^-1 - x)*C[set()] + (-1)*C[{13}] + (-1)*C[{12}] + (-1)*C[{14}] + (x^-1 - x)*C[{12, 13, 14}]",-x^-1 - x,"-(2*x^-1-5*x+2*x^3)*E[frozenset()] - (2-x^2)*E[frozenset({13})] - (2-x^2)*E[frozenset({12})] - (2-x^2)*E[frozenset({14})] + (x^-1-x)*E[frozenset({12, 13, 14})]"
"{0, 11}",2,(-1)*C[set()] + (x^-1 - x)*C[{11}],-1,-(2-x^2)*E[frozenset()] + (x^-1-x)*E[frozenset({11})]
"{0, 10, 11, 14}",3,"(-x^-1 - x)*C[set()] + (-1)*C[{10}] + (-1)*C[{14}] + (-1)*C[{11}] + (x^-1 - x)*C[{10, 11, 14}]",-x^-1 - x,"-(2*x^-1-5*x+2*x^3)*E[frozenset()] - (2-x^2)*E[frozenset({10})] - (2-x^2)*E[frozenset({14})] - (2-x^2)*E[frozenset({11})] + (x^-1-x)*E[frozenset({10, 11, 14})]"
"{0, 11, 12}",3,"(-x^-1 - x)*C[set()] + (-1)*C[{12}] + (-1)*C[{11}] + (x^-1 - x)*C[{11, 12}]",-x^-1 - x,"-(2*x^-1-3*x+x^3)*E[frozenset()] - (2-x^2)*E[frozenset({12})] - (2-x^2)*E[frozenset({11})] + (x^-1-x)*E[frozenset({11, 12})]"


F,rk(F),delta(C[F]),coeff of empty,in natural basis
"{0, 1, 5}",2,"(-1 + x^2)*C[set()] + (1)*C[{1, 5}]",-1 + x^2,"-(2-2*x^2)*E[frozenset()] - x*E[frozenset({5})] - x*E[frozenset({1})] + E[frozenset({1, 5})]"
"{0, 1, 12, 5}",3,"(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{12}] + (1)*C[{1, 12, 5}]",-x^-1 + x,"-(2*x^-1-4*x+2*x^3)*E[frozenset()] - (2-2*x^2)*E[frozenset({12})] - (1-x^2)*E[frozenset({5})] - x*E[frozenset({12, 5})] - (1-x^2)*E[frozenset({1})] - x*E[frozenset({1, 12})] - x*E[frozenset({1, 5})] + E[frozenset({1, 12, 5})]"
"{0, 1, 13, 5}",3,"(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{13}] + (1)*C[{1, 13, 5}]",-x^-1 + x,"-(2*x^-1-4*x+2*x^3)*E[frozenset()] - (2-2*x^2)*E[frozenset({13})] - (1-x^2)*E[frozenset({5})] - x*E[frozenset({13, 5})] - (1-x^2)*E[frozenset({1})] - x*E[frozenset({1, 5})] - x*E[frozenset({1, 13})] + E[frozenset({1, 13, 5})]"
"{0, 2, 6}",2,"(-1 + x^2)*C[set()] + (1)*C[{2, 6}]",-1 + x^2,"-(2-2*x^2)*E[frozenset()] - x*E[frozenset({2})] - x*E[frozenset({6})] + E[frozenset({2, 6})]"
"{0, 2, 10, 6}",3,"(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{10}] + (1)*C[{2, 10, 6}]",-x^-1 + x,"-(2*x^-1-4*x+2*x^3)*E[frozenset()] - (2-2*x^2)*E[frozenset({10})] - (1-x^2)*E[frozenset({2})] - x*E[frozenset({2, 10})] - (1-x^2)*E[frozenset({6})] - x*E[frozenset({2, 6})] - x*E[frozenset({10, 6})] + E[frozenset({2, 10, 6})]"
"{0, 1, 5, 14}",3,"(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{14}] + (1)*C[{1, 5, 14}]",-x^-1 + x,"-(2*x^-1-4*x+2*x^3)*E[frozenset()] - (2-2*x^2)*E[frozenset({14})] - (1-x^2)*E[frozenset({5})] - (1-x^2)*E[frozenset({1})] - x*E[frozenset({1, 5})] - x*E[frozenset({1, 14})] - x*E[frozenset({5, 14})] + E[frozenset({1, 5, 14})]"
"{0, 2, 6, 14}",3,"(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{14}] + (1)*C[{2, 6, 14}]",-x^-1 + x,"-(2*x^-1-4*x+2*x^3)*E[frozenset()] - (2-2*x^2)*E[frozenset({14})] - (1-x^2)*E[frozenset({2})] - (1-x^2)*E[frozenset({6})] - x*E[frozenset({2, 6})] - x*E[frozenset({2, 14})] - x*E[frozenset({6, 14})] + E[frozenset({2, 6, 14})]"
"{0, 1, 5, 12, 13, 14}",4,"(-x^-2 + x^2)*C[set()] + (-x^-1 + x)*C[{13}] + (-x^-1 + x)*C[{12}] + (-x^-1 + x)*C[{14}] + (-1 + x^2)*C[{12, 13, 14}] + (1)*C[{1, 5, 12, 13, 14}]",-x^-2 + x^2,"-(2*x^-2-8+10*x^2-4*x^4)*E[frozenset()] - (2*x^-1-4*x+2*x^3)*E[frozenset({13})] - (2*x^-1-4*x+2*x^3)*E[frozenset({12})] - (2*x^-1-4*x+2*x^3)*E[frozenset({14})] - (2-2*x^2)*E[frozenset({12, 13, 14})] - (x^-1-3*x+2*x^3)*E[frozenset({5})] - (1-x^2)*E[frozenset({13, 5})] - (1-x^2)*E[frozenset({12, 5})] - (x^-1-3*x+2*x^3)*E[frozenset({1})] - (1-x^2)*E[frozenset({1, 12})] - (1-2*x^2)*E[frozenset({1, 5})] - x*E[frozenset({1, 12, 5})] - (1-x^2)*E[frozenset({1, 13})] - x*E[frozenset({1, 13, 5})] - (1-x^2)*E[frozenset({1, 14})] - (1-x^2)*E[frozenset({5, 14})] - x*E[frozenset({1, 5, 14})] - x*E[frozenset({13, 12, 5, 14})] - x*E[frozenset({1, 12, 13, 14})] + E[frozenset({1, 5, 12, 13, 14})]"
"{0, 2, 11, 6}",3,"(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{11}] + (1)*C[{2, 11, 6}]",-x^-1 + x,"-(2*x^-1-4*x+2*x^3)*E[frozenset()] - (2-2*x^2)*E[frozenset({11})] - (1-x^2)*E[frozenset({2})] - (1-x^2)*E[frozenset({6})] - x*E[frozenset({2, 6})] - x*E[frozenset({2, 11})] - x*E[frozenset({11, 6})] + E[frozenset({2, 11, 6})]"
"{0, 2, 6, 10, 11, 14}",4,"(-x^-2 + x^2)*C[set()] + (-x^-1 + x)*C[{10}] + (-x^-1 + x)*C[{14}] + (-x^-1 + x)*C[{11}] + (-1 + x^2)*C[{10, 11, 14}] + (1)*C[{2, 6, 10, 11, 14}]",-x^-2 + x^2,"-(2*x^-2-8+10*x^2-4*x^4)*E[frozenset()] - (2*x^-1-4*x+2*x^3)*E[frozenset({10})] - (2*x^-1-4*x+2*x^3)*E[frozenset({14})] - (2*x^-1-4*x+2*x^3)*E[frozenset({11})] - (2-2*x^2)*E[frozenset({10, 11, 14})] - (x^-1-3*x+2*x^3)*E[frozenset({2})] - (1-x^2)*E[frozenset({2, 10})] - (x^-1-3*x+2*x^3)*E[frozenset({6})] - (1-2*x^2)*E[frozenset({2, 6})] - (1-x^2)*E[frozenset({10, 6})] - x*E[frozenset({2, 10, 6})] - (1-x^2)*E[frozenset({2, 14})] - (1-x^2)*E[frozenset({6, 14})] - x*E[frozenset({2, 6, 14})] - (1-x^2)*E[frozenset({2, 11})] - (1-x^2)*E[frozenset({11, 6})] - x*E[frozenset({2, 11, 6})] - x*E[frozenset({10, 11, 6, 14})] - x*E[frozenset({11, 2, 10, 14})] 

F,rk(F),delta(C[F]),coeff of empty,in natural basis
{2},1,(1)*C[{2}],0,-x*E[frozenset()] + E[frozenset({2})]
"{2, 10}",2,"(1)*C[{2, 10}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({10})] - x*E[frozenset({2})] + E[frozenset({2, 10})]"
{3},1,(1)*C[{3}],0,-x*E[frozenset()] + E[frozenset({3})]
"{3, 13}",2,"(1)*C[{3, 13}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({3})] - x*E[frozenset({13})] + E[frozenset({3, 13})]"
{5},1,(1)*C[{5}],0,-x*E[frozenset()] + E[frozenset({5})]
"{2, 5}",2,"(1)*C[{2, 5}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({2})] - x*E[frozenset({5})] + E[frozenset({2, 5})]"
"{3, 5}",2,"(1)*C[{3, 5}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({3})] - x*E[frozenset({5})] + E[frozenset({3, 5})]"
"{13, 5}",2,"(1)*C[{13, 5}]",0,"-(1-x^2)*E[frozenset()] - x*E[frozenset({13})] - x*E[frozenset({5})] + E[frozenset({13, 5})]"
"{13, 3, 5}",3,"(1)*C[{13, 3, 5}]",0,"-(x^-1-2*x+x^3)*E[frozenset()] - (1-x^2)*E[frozenset({3})] - (1-x^2)*E[frozenset({13})] - x*E[frozenset({3, 13})] - (1-x^2)*E[frozenset({5})] - x*E[frozenset({3, 5})] - x*E[frozenset({13, 5})] + E[frozenset({13, 3, 5})]"
"{2, 3, 12}",2,"(1)*C[{2, 3, 12}]",0,"-(1-2*x^2)*E[frozenset()] - x*E[frozenset({2})] - x*E[frozenset({3})] - x*E[frozenset({12})] + E[frozenset({2, 3, 12})]"


In [7]:
#\Delta(C[F]) - C[F\e] = 
def delta_diff(F):
    return delta(C(F)) -C_dele(F - {e})

print(E_basis.keys()[-1])
delta_diff(E_basis.keys()[-1])


frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})


-(x^-3-10*x^-1+35*x-50*x^3+24*x^5)*E[frozenset()] - (x^-2-6+11*x^2-6*x^4)*E[frozenset({10})] - (x^-2-6+11*x^2-6*x^4)*E[frozenset({13})] - (x^-1-3*x+2*x^3)*E[frozenset({10, 13})] - (x^-2-6+11*x^2-6*x^4)*E[frozenset({12})] - (x^-2-6+11*x^2-6*x^4)*E[frozenset({14})] - (x^-1-3*x+2*x^3)*E[frozenset({12, 13, 14})] - (x^-2-6+11*x^2-6*x^4)*E[frozenset({11})] - (x^-1-3*x+2*x^3)*E[frozenset({10, 11, 14})] - (x^-1-3*x+2*x^3)*E[frozenset({11, 12})] - (x^-2-6+11*x^2-6*x^4)*E[frozenset({9})] - (x^-1-3*x+2*x^3)*E[frozenset({9, 11, 13})] - (x^-1-3*x+2*x^3)*E[frozenset({9, 14})] - (x^-1-3*x+2*x^3)*E[frozenset({9, 10, 12})] - (1-x^2)*E[frozenset({9, 10, 11, 12, 13, 14})]

In [8]:
def delta_diff_comp(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly1 = f(M.contract(G))(x^-2)
        poly2 = f(M.contract(G.union({e})))(x^-2)
        poly3 = f(mdele.contract(G))(x^-2)
        poly_coeff = P(x^(M.rank(ground_set) - M.rank(G)) * (poly1 + x^(-2) * poly2 - poly3))
        print(poly1,', ', poly2,', ', poly3, ', ', poly_coeff)
        sum += Eq_mdele(poly_coeff * E_mdele_basis[G])
    return sum

delta_diff_comp(M, reduced_characteristic_polynomial)

x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-6 - 9*x^-4 + 26*x^-2 - 24 ,  x^-4 - 5*x^-2 + 6 ,  x^-6 - 8*x^-4 + 21*x^-2 - 18 ,  x^-2 - 6 + 11*x^2 - 6*x^4
x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-6 - 9*x^-4 + 26*x^-2 - 24 ,  x^-4 - 5*x^-2 + 6 ,  x^-6 - 8*x^-4 + 21*x^-2 - 18 ,  x^-2 - 6 + 11*x^2 - 6*x^4
x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-6 - 9*x^-4 + 26*x^-2 - 24 ,  x^-4 - 5*x^-2 + 6 ,  x^-6 - 8*x^-4 + 21*x^-2 - 18 ,  x^-2 - 6 + 11*x^2 - 6*x^4
x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-6 - 9*x^-4 + 26*x^-2 - 24 ,  x^-4 - 5*x^-2 + 6 ,  x^-6 - 8*x^-4 + 21*x^-2 - 18 ,  x^-2 - 6 + 11*x^2 - 6*x^4
x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-4 - 5*x^-2 + 6 ,  x^-2 - 2 ,  x^-4 - 4*x^-2 + 4 ,  x^-1 - 3*x + 2*x^3
x^-6 - 9*x^-4 + 26*x^-2 - 24 ,  x^-4 - 5*x^-2

(x^-3-10*x^-1+35*x-50*x^3+24*x^5)*E[frozenset()] + (x^-2-6+11*x^2-6*x^4)*E[frozenset({11})] + (x^-2-6+11*x^2-6*x^4)*E[frozenset({14})] + (x^-1-3*x+2*x^3)*E[frozenset({10, 11, 14})] + (x^-1-3*x+2*x^3)*E[frozenset({11, 12})] + (x^-2-6+11*x^2-6*x^4)*E[frozenset({9})] + (x^-1-3*x+2*x^3)*E[frozenset({9, 14})] + (x^-1-3*x+2*x^3)*E[frozenset({9, 11, 13})] + (x^-2-6+11*x^2-6*x^4)*E[frozenset({10})] + (x^-2-6+11*x^2-6*x^4)*E[frozenset({13})] + (x^-1-3*x+2*x^3)*E[frozenset({10, 13})] + (x^-2-6+11*x^2-6*x^4)*E[frozenset({12})] + (x^-1-3*x+2*x^3)*E[frozenset({12, 13, 14})] + (x^-1-3*x+2*x^3)*E[frozenset({9, 10, 12})] + (1-x^2)*E[frozenset({9, 10, 11, 12, 13, 14})]